# Federal CTC Expansion Reduces MA State Tax

This notebook demonstrates how federal CTC expansion reduces Massachusetts state tax for household 4428.

In [1]:
from policyengine_us import Microsimulation
from policyengine_us.model_api import *
import numpy as np

DATASET = "hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5"
YEAR = 2026


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/homebrew/Cellar/python@3.10/3.10.17/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/homebrew/Cellar/python@3.10/3.10.17/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/maxghenis/Library/Python/3.10/lib/python/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/maxg

AttributeError: _ARRAY_API not found

## Compare Scenarios

We'll compare the same reform stack with and without CTC expansion:

In [2]:
# Reform stack WITHOUT CTC
reform_no_ctc = Reform.from_dict({
    "gov.irs.income.bracket.rates.2": {"2026-01-01": 0.15},
    "gov.irs.income.bracket.rates.3": {"2026-01-01": 0.25},
    "gov.irs.income.bracket.rates.4": {"2026-01-01": 0.28},
    "gov.irs.deductions.standard.amount.JOINT": {"2026-01-01": 48900},
    "gov.irs.income.exemption.amount": {"2026-01-01": 0},
}, country_id="us")

sim_no_ctc = Microsimulation(reform=reform_no_ctc, dataset=DATASET)

# Find tax unit 442801
tax_unit_ids = sim_no_ctc.calculate("tax_unit_id", period=YEAR).values
tu_idx = np.where(tax_unit_ids == 442801)[0][0]

print("WITHOUT CTC EXPANSION:")
print(f"  Federal itemizes: {bool(sim_no_ctc.calculate('tax_unit_itemizes', YEAR).values[tu_idx])}")
print(f"  MA state tax: ${sim_no_ctc.calculate('ma_income_tax', YEAR).values[tu_idx]:,.2f}")
print(f"  MA Part B exemption: ${sim_no_ctc.calculate('ma_part_b_taxable_income_exemption', YEAR).values[tu_idx]:,.2f}")

WITHOUT CTC EXPANSION:


  Federal itemizes: False


  MA state tax: $-4,409.01
  MA Part B exemption: $12,800.00


In [3]:
# Add CTC expansion
reform_with_ctc = Reform.from_dict({
    "gov.irs.income.bracket.rates.2": {"2026-01-01": 0.15},
    "gov.irs.income.bracket.rates.3": {"2026-01-01": 0.25},
    "gov.irs.income.bracket.rates.4": {"2026-01-01": 0.28},
    "gov.irs.deductions.standard.amount.JOINT": {"2026-01-01": 48900},
    "gov.irs.income.exemption.amount": {"2026-01-01": 0},
    # CTC expansion
    "gov.contrib.reconciliation.ctc.in_effect": {"2026-01-01": True},
    "gov.irs.credits.ctc.amount.base[0].amount": {"2026-01-01": 2200},
}, country_id="us")

sim_with_ctc = Microsimulation(reform=reform_with_ctc, dataset=DATASET)

print("WITH CTC EXPANSION:")
print(f"  Federal itemizes: {bool(sim_with_ctc.calculate('tax_unit_itemizes', YEAR).values[tu_idx])}")
print(f"  MA state tax: ${sim_with_ctc.calculate('ma_income_tax', YEAR).values[tu_idx]:,.2f}")
print(f"  MA Part B exemption: ${sim_with_ctc.calculate('ma_part_b_taxable_income_exemption', YEAR).values[tu_idx]:,.2f}")

WITH CTC EXPANSION:


  Federal itemizes: True


  MA state tax: $-4,465.42
  MA Part B exemption: $13,928.17


## The Mechanism

In [4]:
# Calculate the impact
ma_tax_change = (sim_with_ctc.calculate('ma_income_tax', YEAR).values[tu_idx] - 
                 sim_no_ctc.calculate('ma_income_tax', YEAR).values[tu_idx])

exemption_change = (sim_with_ctc.calculate('ma_part_b_taxable_income_exemption', YEAR).values[tu_idx] - 
                    sim_no_ctc.calculate('ma_part_b_taxable_income_exemption', YEAR).values[tu_idx])

print("IMPACT OF FEDERAL CTC EXPANSION ON MA STATE TAX:")
print(f"  MA state tax reduction: ${-ma_tax_change:,.2f}")
print(f"  MA Part B exemption increase: ${exemption_change:,.2f}")

print(f"\nMECHANISM:")
print(f"1. CTC expansion triggers federal itemization")
print(f"2. MA allows itemizers to claim medical expense deduction")
print(f"3. Medical expense deduction: ${sim_with_ctc.calculate('medical_expense_deduction', YEAR).values[tu_idx]:,.2f}")
print(f"4. MA tax rate: 5%")
print(f"5. Tax savings: ${exemption_change:,.2f} × 5% = ${exemption_change * 0.05:,.2f}")

# Verify federal tax is same either way
fed_tax_item = sim_with_ctc.calculate('tax_liability_if_itemizing', YEAR).values[tu_idx]
fed_tax_std = sim_with_ctc.calculate('tax_liability_if_not_itemizing', YEAR).values[tu_idx]
print(f"\nFederal tax is same either way: {abs(fed_tax_item - fed_tax_std) < 0.01}")

IMPACT OF FEDERAL CTC EXPANSION ON MA STATE TAX:
  MA state tax reduction: $56.41
  MA Part B exemption increase: $1,128.17

MECHANISM:
1. CTC expansion triggers federal itemization
2. MA allows itemizers to claim medical expense deduction
3. Medical expense deduction: $1,128.17
4. MA tax rate: 5%
5. Tax savings: $1,128.17 × 5% = $56.41

Federal tax is same either way: True
